<a href="https://colab.research.google.com/github/nithishhhhhhh/Business_Chatbot/blob/main/Business_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28 pinecone-client pymupdf


In [ ]:
import openai
import pinecone
import os
import fitz

In [37]:
openai.api_key = 'personal_openai_key'
pinecone_api_key = 'pinecone_key'
pinecone_env = 'us-east-1'

In [ ]:
client = pinecone.Pinecone(api_key=pinecone_api_key, environment=pinecone_env)
index_name = 'businesschatbot'

In [ ]:
index = client.Index(index_name)

In [ ]:
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

In [ ]:
def get_embedding(text):
    response = openai.Embedding.create(input=[text], model="text-embedding-ada-002")
    return response['data'][0]['embedding']

In [ ]:
pdf_directory = '/content/file'

In [ ]:
for more in os.listdir(pdf_directory):
    if more.endswith('.pdf'):
        file_path = os.path.join(pdf_directory, more)
        document_text = extract_text_from_pdf('/content/file/More.pdf')
        doc_id = os.path.splitext(more)[0]
        embedding = get_embedding(document_text)
        index.upsert([(doc_id, embedding)])


As i dont have Upgraded OpenAi Plan. I can't do embedding step.

In [ ]:
def retrieve_documents(query, top_k=5):
    query_embedding = get_embedding(query)
    results = index.query(queries=[query_embedding], top_k=top_k)
    return results['matches']

In [ ]:
def generate_answer(query, retrieved_docs):
    context = " ".join([doc["metadata"]["text"] for doc in retrieved_docs])
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Context: {context}\n\nQuestion: {query}\nAnswer:",
        max_tokens=150
    )
    return response['choices'][0]['text']

In [ ]:
query = "When was More founded?"
retrieved_docs = retrieve_documents(query)
answer = generate_answer(query, retrieved_docs)
print(f"Question: {query}")
print(f"Answer: {answer}")

Question: When was More founded?





































Answer: 2007


